In [1]:
import os
import glob
import seaborn as sns
from astropy.io import fits
import stsci.image as image
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")
%load_ext lab_black
%config InlineBackend.figure_format = 'retina'

In [2]:
class Image:

    # construct with an 2D np.array
    def __init__(self, arrays):
        self.data = arrays

    @classmethod

    # construct with .fits files reading by astropy.io.fits
    def read_fits(cls, input_file_format):
        fits_list = []

        # get all files matches the file_format, eg. "bias*.fits" or "bias1.fits"
        for fits_image_filename in glob.glob(input_file_format):
            with fits.open(fits_image_filename) as hdul:
                fits_list.append(cls(hdul[0].data))

        if len(fits_list) == 1:
            # if only one file, return a Image instead of a np.array
            return np.array(fits_list)[0]

        else:
            # if more than one file, return a np.array consisted of Images
            return np.array(fits_list)

    # reconstruct the operators
    def __sub__(self, other):
        return Image(self.data - other.data)

    def __add__(self, other):
        return Image(self.data + other.data)

    def __mul__(self, other):
        return Image(self.data * other.data)

    def __truediv__(self, other):
        return Image(self.data / other.data)

    def normalize(self):
        return Image(self.data / np.mean(self.data))

    # plot a heatmap with seaborn
    def plot(self):
        plt.figure(figsize=(10, 6))
        sns.heatmap(self.data)

In [3]:
def combine():

    # List of images to combine
    input_file = input("  input = ")

    # List of output images
    output_file = input(" output = ")

    # Type of combine operation
    combine_mode = input("combine = ")

    # get all files matches the file_format, eg. "bias*.fits" or "bias1.fits"
    data_to_combine = [Image.read_fits(file).data for file in glob.glob(input_file)]

    if combine_mode == "median":
        combined_image = image.median(data_to_combine)

    elif combine_mode == "average":
        combined_image = image.average(data_to_combine)

    if os.path.isfile(output_file):
        print("File '%s' already exists." % output_file)
    else:
        # save combined image as output_file
        fits.HDUList([fits.PrimaryHDU(combined_image)]).writeto(output_file)

# Working directory

In [4]:
os.chdir("/Users/jackarroy/Downloads/test_CCD_reduction")

# Combine bias

In [5]:
combine()

  input =  bias*.fits
 output =  master_bias_py.fits
combine =  median


# Subtract bias

In [6]:
bias = Image.read_fits("master_bias_py.fits")

flat = Image.read_fits("flat*.fits")

m67 = Image.read_fits("m67*.fits")

flat -= bias

m67 -= bias

# Combine flat

In [7]:
combine()

  input =  flat_b*.fits
 output =  master_flat_b_py.fits
combine =  average


# Normalize flat

In [8]:
flat_b = Image.read_fits("master_flat_b_py.fits")

flat_b = Image.normalize(flat_b)

# Divided by flat

In [9]:
m67_b = Image.read_fits("m67_b*.fits")

m67_b /= flat_b

# Combine target image

In [10]:
combine()

  input =  m67_b*.fits
 output =  m67_b_final_py.fits
combine =  average


In [11]:
m67_b_final_py = Image.read_fits("m67_b_final_py.fits")
m67_b_final_py.data

array([[  21.638529,   22.945717,   23.168673, ...,   18.559359,
          18.118979,   21.08277 ],
       [  21.933144,   21.02201 ,   19.70898 , ...,   19.561827,
          18.481653,   22.101898],
       [  19.762846,   19.333054,   19.682869, ...,   18.055632,
          21.93515 ,   22.454481],
       ...,
       [  21.633627,   20.00421 ,   19.739054, ...,   16.223099,
          20.599184,   18.256285],
       [  20.55961 ,   18.661057,   20.729004, ...,   23.314104,
          22.139015,   22.600708],
       [1021.3212  , 2845.4563  , 3016.7693  , ..., 6951.172   ,
        5632.7827  , 3328.0964  ]], dtype=float32)

In [12]:
m67_b_final = Image.read_fits("m67_b_final.fits")
m67_b_final.data

array([[  21.638529,   22.945717,   23.168673, ...,   18.559359,
          18.118979,   21.08277 ],
       [  21.933144,   21.02201 ,   19.70898 , ...,   19.561827,
          18.481653,   22.101898],
       [  19.762846,   19.333054,   19.682869, ...,   18.055632,
          21.93515 ,   22.454481],
       ...,
       [  21.633627,   20.00421 ,   19.739054, ...,   16.223099,
          20.599184,   18.256285],
       [  20.55961 ,   18.661057,   20.729004, ...,   23.314104,
          22.139015,   22.600708],
       [1021.3212  , 2845.4563  , 3016.7693  , ..., 6951.172   ,
        5632.7827  , 3328.0964  ]], dtype=float32)

In [15]:
False in (m67_b_final.data == m67_b_final_py.data)

False